<a href="https://colab.research.google.com/github/arunshar/RAGPytorch/blob/main/LocalRag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [3]:
!git clone https://github.com/arunshar/RAGPytorch.git

Cloning into 'RAGPytorch'...


In [1]:
#perform Google Colab installs (if runnning in Google Colab)
import os

if "COLAB_GPU" in os.environ:
  print("[INFO] Running in Google Colab, installing requirements.")
  !pip install -U torch
  !pip install PyMuPDF
  !pip install tqdm
  !pip install sentence-transformers
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 613.1 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.4 MB/s eta 0:

In [3]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print ("File does not exists, downloading...")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path

  response = requests.get(url)

  if response.status_code == 200:
    with open(filename, "wb") as file:
      file.write(response.content)

    print(f"The file has been downloaded and saved as {filename}")
  else:
    print(f"Failed to download the file. Status code: {response.status_code}")

else:
  print(f"File {pdf_path} exists.")

File does not exists, downloading...
The file has been downloaded and saved as human-nutrition-text.pdf


In [17]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:

    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
      text = page.get_text()
      text = text_formatter(text)
      pages_and_texts.append({"page_number": page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                              "page_token_count": len(text)/4,
                            "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [18]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 1072,
  'page_char_count': 1234,
  'page_word_count': 199,
  'page_sentence_count_raw': 9,
  'page_token_count': 308.5,
  'text': 'The Health At Every Size® Principles are:  1. Weight Inclusivity: Accepting and respecting the diversity of  body shapes and sizes  2. Health Enhancement: Recognizing that health and well-being  are multi-dimensional and that they include physical, social,  spiritual, occupational, emotional, and intellectual aspects  3. Respectful Care: Promoting all aspects of health and well- being for people of all sizes  4. Eating for Well-being: Promoting eating in a manner which  balances individualized nutritional needs, hunger, satiety,  appetite, and pleasure  5. Life-Enhancing Movement: Promoting individually  appropriate, enjoyable, life-enhancing physical activity, rather  than exercise that is focused on a goal of weight loss  In the Hawaiian language, Lokahi means “unity, agreement, accord,  and harmony”.23\xa0 The concept of Lokahi can be us

In [20]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [21]:
#Get stats
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,9.97,287.15
std,348.86,560.44,95.75,6.19,140.11
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.75,134.00,4.00,190.69
50%,562.50,1232.50,215.00,10.00,308.12
75%,864.25,1605.25,271.25,14.00,401.31
max,1166.00,2308.00,429.00,32.00,577.00


In [23]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2
list(doc.sents)

[This is a sentence., This another sentence.]

In [24]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  item["sentences"] = [str(sentence) for sentence in item["sentences"]]

  item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [25]:
random.sample(pages_and_texts, k=1)

[{'page_number': 6,
  'page_char_count': 1806,
  'page_word_count': 313,
  'page_sentence_count_raw': 16,
  'page_token_count': 451.5,
  'text': 'down digestible complex carbohydrates to simple sugars, mostly  glucose. Glucose is then transported to all our cells where it is  stored, used to make energy, or used to build macromolecules. Fiber  is also a complex carbohydrate, but it cannot be broken down by  digestive enzymes in the human intestine. As a result, it passes  through the digestive tract undigested unless the bacteria that  inhabit the colon or large intestine break it down.  One gram of digestible carbohydrates yields four kilocalories of  energy for the cells in the body to perform work. In addition to  providing energy and serving as building blocks for bigger  macromolecules, carbohydrates are essential for proper functioning  of the nervous system, heart, and kidneys. As mentioned, glucose  can be stored in the body for future use. In humans, the storage  molecule of c

In [26]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.59,198.89,9.97,287.15,10.32
std,348.86,560.44,95.75,6.19,140.11,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.75,134.00,4.00,190.69,5.00
50%,562.50,1232.50,215.00,10.00,308.12,10.00
75%,864.25,1605.25,271.25,14.00,401.31,15.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00
